In [ ]:
import numpy as np

def balanced_extremes_aggregator(n, observed_ratings, q):
    """
    Implements the Balanced Extremes Aggregator (BEA) for rating aggregation
    with known sample size.

    Args:
        n: The total number of raters (sample size).
        observed_ratings: A list or numpy array of observed ratings.  Ratings
            should be integers.  Missing ratings should *not* be included in
            this list (i.e., this list contains only the *observed* ratings).
        q: The lower bound on the reporting probability for any rating value.

    Returns:
        The aggregated rating according to BEA.
    """
    observed_ratings = np.array(observed_ratings)
    m = np.max(observed_ratings)  # Find the maximum rating value
    no = len(observed_ratings)     # Number of observed ratings
    nu = n - no                    # Number of unobserved ratings

    if no == 0: # Handle the case where there is no observation.
        return (1 + m) / 2

    # Calculate the counts of the extreme ratings (1 and m)
    n1 = np.sum(observed_ratings == 1)
    nm = np.sum(observed_ratings == m)

    # Calculate the empirical average of the observed ratings
    mu_o = np.mean(observed_ratings)

    # Calculate the parameter alpha (using a simplified version for demonstration)
    # The full version involves solving a maximization, which is complex.  This
    # simplified version captures the *spirit* of BEA.
    alpha = (n1 + q * (n - n1 - nm)) / (no + q * (n- no))  #Simplified. The real Alpha is different.

    # Estimate the mean of the unobserved ratings
    mu_u = alpha * 1 + (1 - alpha) * m

    # Calculate the final aggregated rating
    f_bea = (no * mu_o + nu * mu_u) / n
    return f_bea


def polarizing_averaging_aggregator(observed_ratings, q):
    """
    Implements the Polarizing-Averaging Aggregator (PAA) for rating
    aggregation with unknown sample size.

    Args:
        observed_ratings: A list or numpy array of observed ratings. Ratings
            should be integers.
        q: The lower bound on the reporting probability for any rating value.

    Returns:
        The aggregated rating according to PAA.
    """
    observed_ratings = np.array(observed_ratings)
    m = int(np.max(observed_ratings))  # Maximum rating value, ensure integer
    
    if len(observed_ratings) == 0:
        return (1+m)/2

    # Create the empirical distribution (normalized histogram)
    counts = np.bincount(observed_ratings)[1:]  # Ignore counts of 0 (if any)
    empirical_dist = counts / np.sum(counts)

    # Function to calculate k1 and k2 (thresholds)
    def calculate_thresholds(p):
        k1 = 1
        for k in range(1, m):
            sum1 = np.sum(np.arange(1, k+1) * p[:k])
            sum2 = q * np.sum(np.arange(k + 1, m + 1) * p[k:])
            if sum1 - k*np.sum(p[:k]) + sum2 - k* q * np.sum(p[k:]) >= 0:
                k1 = k
                break
        
        k2 = 1
        for k in range(1, m):
          sum1 = q* np.sum(np.arange(1,k+1)*p[:k])
          sum2 = np.sum(np.arange(k+1, m+1) *p[k:])
          if q * sum1 - k* q*np.sum(p[:k]) + sum2 - k*np.sum(p[k:]) >= 0:
            k2=k
            break
        return k1, k2

    k1, k2 = calculate_thresholds(empirical_dist)
    
    # Create the two modified histograms
    modified_dist1 = np.copy(empirical_dist)
    modified_dist1[k1:] *= q   # Multiply counts above k1 by q

    modified_dist2 = np.copy(empirical_dist)
    modified_dist2[:k2] *= q    # Multiply counts below k2 by q

    # Calculate the means of the modified distributions
    l_p = np.sum(np.arange(1, m + 1) * modified_dist1)
    u_p = np.sum(np.arange(1, m + 1) * modified_dist2)
    
    # Calculate the final aggregated rating
    f_paa = (l_p + u_p) / 2
    return f_paa



def simple_averaging(observed_ratings):
    """
    Calculates the simple average of the observed ratings.

    Args:
        observed_ratings: A list or numpy array of observed ratings.

    Returns:
        The simple average of the ratings.
    """
    if len(observed_ratings) == 0:
        return np.nan  # Return NaN if there are no observed ratings
    return np.mean(observed_ratings)
    

In [6]:
print("Example 1: BEA (Known sample size)")
n = 20  # Total number of raters
observed_ratings_bea = [1, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]  # 15 observed
q_bea = 0.3  # Lower bound on reporting probability
bea_result = balanced_extremes_aggregator(n, observed_ratings_bea, q_bea)
print(f"BEA Result: {bea_result}")
average_result = simple_averaging(observed_ratings_bea)
print(f"Simple Averaging Result: {average_result}")
print()

print("Example 2: PAA (Unknown sample size)")
observed_ratings_paa = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 5, 5, 5]
q_paa = 0.3  # Lower bound on reporting probability
paa_result = polarizing_averaging_aggregator(observed_ratings_paa, q_paa)
print(f"PAA Result: {paa_result}")
average_result_paa = simple_averaging(observed_ratings_paa)
print(f"Simple Averaging Result: {average_result_paa}")
print()


print("Example 3: PAA with various ratings")
observed_ratings_paa_mixed = [1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5]
q_paa_mixed = 0.4
paa_result_mixed = polarizing_averaging_aggregator(observed_ratings_paa_mixed, q_paa_mixed)
print(f"PAA Result (mixed ratings): {paa_result_mixed}")
avg_result_mixed = simple_averaging(observed_ratings_paa_mixed)
print(f"Simple Averaging Result (mixed ratings): {avg_result_mixed}")

Example 1: BEA (Known sample size)
BEA Result: 3.6060606060606064
Simple Averaging Result: 3.6666666666666665
Example 2: PAA (Unknown sample size)
PAA Result: 1.5166666666666666
Simple Averaging Result: 2.3333333333333335
Example 3: PAA with various ratings
PAA Result (mixed ratings): 2.3153846153846156
Simple Averaging Result (mixed ratings): 3.3076923076923075
